 Import statements

In [16]:
import cv2
from pytesseract import image_to_string
from PIL import Image 
import numpy as np

This bit separates the crossword itself from the screenshot as grid.png

In [8]:
#Where is our image?
image_path = 'Screenshot.png'

#Preparing for cropping
im = cv2.imread(image_path)
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im = cv2.GaussianBlur(im, (5, 5), 0)
im = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 4)
cv2.imwrite("preprocess.jpg", im)

# Grid is the largest contour
contours, _ = cv2.findContours(im, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
max_area = -1
max_idx = -1
for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if max_area < area < (im.shape[0] * im.shape[1] / 2):
        max_area = area
        max_idx = i

#Saving grid.png
rect = cv2.boundingRect(contours[max_idx])
print(rect)
Image.open(image_path).crop((rect[0], rect[1], rect[0] + rect[2], rect[1] + rect[3])).save("grid.png")


(32, 25, 730, 735)


Crossword hint extraction

In [ ]:
def extract_hints_from_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale for OCR
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use pytesseract to extract text from the image
    extracted_text = str(image_to_string(gray))
    
    # Only consider text after "Prize crossword"
    if "Prize crossword" in extracted_text:
        extracted_text = extracted_text.split("Prize crossword")[1].strip()
    
    # Split the text based on 'Across' and 'Down' to separate the hints
    across_hints, down_hints = extracted_text.split("DOWN")
    across_hints = across_hints.replace("ACROSS", "").strip()

    return across_hints, down_hints

extract_hints_from_image(image_path)

This bit finds the green squares and gets coordinates for them
## Do we need this?

In [15]:
crossword_path = 'grid.png'

def detect_green_squares_and_overlay(crossword_path):
    # Load the image
    image = cv2.imread(crossword_path)
    
    # Convert to HSV for color-based detection
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for green color
    lower_green = (40, 25, 25)
    upper_green = (45, 255,255)

    
    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv, lower_green, upper_green)
    
    # Convert the binary mask to a 3-channel image
    mask_colored = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # Overlay the mask on the original image
    alpha = 0.5  # Define the opacity for overlay
    overlay = cv2.addWeighted(image, 1 - alpha, mask_colored, alpha, 0)
    
    # Find contours to detect squares
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Placeholder list to store positions of green squares
    positions = []
    
    for contour in contours:
        # Compute the center of the contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
            # Ignore the center element (based on contour area, adjust the threshold as needed)
            if cv2.contourArea(contour) < 5000:  # 5000 is an example threshold, adjust as necessary
                positions.append((cX, cY))
                
                # Draw a circle at the detected position on the overlay
                cv2.circle(overlay, (cX, cY), 5, (0, 0, 255), -1)  # Drawing in red for visibility

    # Display the overlay
    cv2.imshow("Overlay", overlay)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    print(type(positions))
    print(positions)
    return positions
    

detect_green_squares_and_overlay(crossword_path)



<class 'list'>
[(308, 703), (85, 591), (587, 422), (252, 254), (29, 29)]


[(308, 703), (85, 591), (587, 422), (252, 254), (29, 29)]

Experimental bit with the following steps:
1. Use contours to detect every square
2. Get coordinates for every square
3. Based on squares per X coordinates+-2/3 get the size and shape of grid
4. Read contents of every square one by one, and based on findings append symbol to matrix
5. Return matrix

In [22]:
import cv2
import numpy as np
import os

# Load the image
img = cv2.imread('grid.png')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding
adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Use morphological operations to clean up the image
kernel = np.ones((5, 5), np.uint8)
cleaned_thresh = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)

# Divide the image into a 13x13 grid to get the coordinates of each cell
cell_width = img.shape[1] // 13
cell_height = img.shape[0] // 13
grid_coordinates = [(j*cell_width, i*cell_height, cell_width, cell_height) for i in range(13) for j in range(13)]

# Analyze each cell to determine its type (black, white, numbered, green)
# Function to determine the content of a square based on its mean pixel value and presence of small details (numbers)
def determine_square_content_v2(roi_gray, roi_thresh):
    mean_val = np.mean(roi_gray)
    
    # Calculate the ratio of white pixels in the thresholded ROI
    white_ratio = np.sum(roi_thresh == 255) / (roi_thresh.shape[0] * roi_thresh.shape[1])
    
    # If the mean value is low and the white ratio is low, it's likely a black square
    if mean_val < 50 and white_ratio < 0.2:
        return 'X'
    # If the mean value is high and the white ratio is high, it's a white square
    elif mean_val > 200 and white_ratio > 0.75:
        # Check for numbers in the square
        contours, _ = cv2.findContours(roi_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) < 0.15 * roi_thresh.size:
            return "(num)"  # Placeholder for numbered square
        else:
            return 'O'
    # If the mean value is between the thresholds, it's likely a green square
    else:
        # Check for numbers in the green square
        contours, _ = cv2.findContours(roi_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) < 0.15 * roi_thresh.size:
            return "(numG)"  # Placeholder for numbered green square
        else:
            return 'G'

text_matrix = []
row = []
for i, coord in enumerate(grid_coordinates):
    x, y, w, h = coord
    roi_gray = gray[y:y+h, x:x+w]
    _, roi_thresh = cv2.threshold(roi_gray, 127, 255, cv2.THRESH_BINARY)
    content = determine_square_content_v2(roi_gray, roi_thresh)
    row.append(content)
    # Check if we've processed 13 squares (one row)
    if (i + 1) % 13 == 0:
        text_matrix.append(row)
        row = []

text_matrix


[['G', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'X', 'O', 'X', 'X', 'O', 'O', 'O', 'X', 'O', 'X', 'O', 'X'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'X', 'O', 'O', 'O', 'O'],
 ['O', 'X', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X'],
 ['X', 'O', 'X', 'O', 'G', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'X'],
 ['O', 'O', 'O', 'O', 'O', 'G', 'G', 'G', 'O', 'O', 'O', 'O', 'O'],
 ['X', 'O', 'X', 'X', 'O', 'G', 'G', 'O', 'O', 'X', 'X', 'O', 'X'],
 ['O', 'O', 'O', 'O', 'O', 'G', 'G', 'O', 'O', 'O', 'G', 'O', 'O'],
 ['X', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'X'],
 ['X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'X', 'O'],
 ['O', 'G', 'O', 'O', 'X', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'O', 'X', 'X', 'O', 'X', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'G', 'X', 'O', 'O', 'O', 'O', 'O', 'O']]

This saves every single cell as a separate image for OCR

In [ ]:
# Function to save every single square except the middle 3x3 squares
save_dir = "numbered_squares/"
def save_all_squares(img, coords):
    saved_files_all = []  # To keep track of saved files
    for i in range(13):
        for j in range(13):
            # Ignoring the middle 3x3 squares
            if 5 <= i <= 7 and 5 <= j <= 7:
                continue
            idx = i*13 + j
            x, y, w, h = coords[idx]
            roi = img[y:y+h, x:x+w]
            
            # Using direct string concatenation for filename
            filename = save_dir + f"all_squares_{i}_{j}.png"
            
            # Attempt to save the image and raise an exception if saving fails
            if not cv2.imwrite(filename, roi):
                raise Exception(f"Could not write image at position {i}, {j} to {filename}")
            
            saved_files_all.append(filename)
    return saved_files_all  # Return the list of saved files


# Save every square except the middle 3x3 ones
saved_files_all = save_all_squares(img, grid_coordinates)

saved_files_all  # Display the first 5 saved file paths for verification


This generates the OCR matrix with numbers and their locations

In [21]:
import pytesseract
from PIL import Image

# Initialize the matrix
ocr_matrix = [[0 for _ in range(13)] for _ in range(13)]

# Process saved squares using pytesseract
for i in range(13):
    for j in range(13):
        # Skip the middle 3x3 squares
        if 5 <= i <= 7 and 5 <= j <= 7:
            ocr_matrix[i][j] = 'X'
            continue
        
        filename = f"numbered_squares/all_squares_{i}_{j}.png"
        
        # Open the image and pre-crop it
        image = Image.open(filename)
        cropped_image = image.crop((4, 3, 25, 27))
        
        # Use pytesseract to extract text from the cropped image
        result = pytesseract.image_to_string(cropped_image, config='--psm 6').strip()
        print(result)
        # If a number is detected, update the matrix with that number
        if result.isdigit():
            ocr_matrix[i][j] = int(result)

# Print the resulting matrix
for row in ocr_matrix:
    print(row)


1
2

3

4

5

6



7



8

9



10









11

12
13

14


15


16







17

18
19
20



21
22
23








24
25



26

27




28




29



30



[1, 0, 2, 0, 0, 3, 0, 4, 0, 5, 0, 6, 0]
[0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 9, 0, 0, 0, 0, 0, 10, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 11, 0, 12, 13, 0, 0, 0, 14, 0, 0, 0, 0]
[15, 0, 0, 0, 0, 'X', 'X', 'X', 16, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 'X', 'X', 'X', 0, 0, 0, 0, 0]
[17, 0, 0, 18, 0, 'X', 'X', 'X', 19, 20, 0, 0, 0]
[0, 0, 0, 21, 0, 22, 0, 23, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24]
[25, 0, 0, 0, 0, 0, 0, 26, 0, 0, 27, 0, 0]
[0, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0]
[29, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0]


This will join them together
## Testing still

In [39]:
# print(text_matrix)
# 
# print('SPAAAAAAAACE')
# 
# print(ocr_matrix)

# Create a deep copy of text_matrix to avoid modifying the original matrix
combined_text_matrix = [row.copy() for row in text_matrix]

# Modified merging approach based on the updated requirements
for i in range(13):
    for j in range(13):
        # For the central 3x3 grid, always "X"
        if 4 <= i <= 6 and 4 <= j <= 6:
            combined_text_matrix[i][j] = "X"
        # For white squares with numbers
        elif combined_text_matrix[i][j] == "O" and ocr_matrix[i][j] != 0:
            combined_text_matrix[i][j] = f"({ocr_matrix[i][j]})"
        # For green squares with numbers
        elif combined_text_matrix[i][j] == "G" and ocr_matrix[i][j] != 0:
            combined_text_matrix[i][j] = f"({ocr_matrix[i][j]}G)"
        # For green squares without numbers
        elif combined_text_matrix[i][j] == "G" and ocr_matrix[i][j] == 0:
            combined_text_matrix[i][j] = "G"
        # For black squares
        elif combined_text_matrix[i][j] == "X":
            combined_text_matrix[i][j] = "X"

print(combined_text_matrix)



intended_matrix = [
    ['(1G)', 'O', '(2)', 'O', 'O', '(3)', 'X', '(4)', 'O', '(5)', 'O', '(6)', 'O'],
    ['O', 'X', 'O', 'X', 'X', '(7)', 'O', 'O', 'X', 'O', 'X', 'O', 'X'],
    ['(8)', 'O', 'O', '(9)', 'O', 'O', 'X', 'O', 'X', 'O', 'X', '(10)', 'O'],
    ['O', 'X', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X'],
    ['X', '(11)', 'X', '(12)', '(13G)', 'O', 'O', 'O', '(14)', 'O', 'X', 'O', 'X'],
    ['(15)', 'O', 'O', 'O', 'O', 'X', 'X', 'X', '(16)', 'O', 'O', 'O', 'O'],
    ['X', 'O', 'X', 'X', 'O', 'X', 'X', 'X', 'O', 'X', 'X', 'O', 'X'],
    ['(17)', 'O', 'O', '(18)', 'O', 'X', 'X', 'X', '(19)', '(20)', 'G', 'O', 'O'],
    ['X', 'O', 'X', '(21)', 'O', '(22)', 'O', '(23)', 'O', 'O', 'X', 'O', 'X'],
    ['X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'X', '(24)'],
    ['(25)', 'G', 'O', 'O', 'X', 'O', 'X', 'O', '(26)', 'O', 'O', '(27)', 'O'],
    ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'O', 'X', 'X', 'O', 'X', 'O'],
    ['(29)', 'O', 'O', 'O', 'O', 'G', 'X', '(30)', 'O', 'O', 'O', 'O', 'O']
]



[['(1G)', 'O', '(2)', 'O', 'O', '(3)', 'X', '(4)', 'O', '(5)', 'O', '(6)', 'O'], ['O', 'X', 'O', 'X', 'X', '(7)', 'O', 'O', 'X', 'O', 'X', 'O', 'X'], ['(8)', 'O', 'O', '(9)', 'O', 'O', 'X', 'O', 'X', '(10)', 'O', 'O', 'O'], ['O', 'X', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X'], ['X', '(11)', 'X', '(12)', 'X', 'X', 'X', 'O', '(14)', 'O', 'X', 'O', 'X'], ['(15)', 'O', 'O', 'O', 'X', 'X', 'X', '(XG)', '(16)', 'O', 'O', 'O', 'O'], ['X', 'O', 'X', 'X', 'X', 'X', 'X', '(X)', 'O', 'X', 'X', 'O', 'X'], ['(17)', 'O', 'O', '(18)', 'O', '(XG)', '(XG)', '(X)', '(19)', '(20)', 'G', 'O', 'O'], ['X', 'O', 'X', '(21)', 'O', '(22)', 'O', '(23)', 'O', 'O', 'X', 'O', 'X'], ['X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X', 'X', '(24)'], ['(25)', 'G', 'O', 'O', 'X', 'O', 'X', '(26)', 'O', 'O', '(27)', 'O', 'O'], ['X', 'O', 'X', 'O', 'X', '(28)', 'O', 'O', 'X', 'X', 'O', 'X', 'O'], ['(29)', 'O', 'O', 'O', 'O', 'G', 'X', '(30)', 'O', 'O', 'O', 'O', 'O']]


This will test if our manually made matrix matches the automated matrix

In [41]:
def test_matrices(matrix1, matrix2):
    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            if matrix1[i][j] != matrix2[i][j]:
                return False
    return True

# Test
result = test_matrices(combined_text_matrix, intended_matrix)
print(result)
print("Matrices are identical:" if result else "Matrices are not identical.")


False
Matrices are not identical.


Comparing matrices to see what's up

In [42]:
def find_differences(matrix1, matrix2):
    differences = []
    
    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            if matrix1[i][j] != matrix2[i][j]:
                differences.append({
                    "coordinates": (i, j),
                    "matrix1_value": matrix1[i][j],
                    "matrix2_value": matrix2[i][j]
                })
                
    return differences

differences = find_differences(combined_text_matrix, intended_matrix)

for diff in differences:
    print(f"Coordinates: {diff['coordinates']}")
    print(f"combined_text_matrix value: {diff['matrix1_value']}")
    print(f"intended_matrix value: {diff['matrix2_value']}")
    print("--------------------")


Coordinates: (2, 9)
combined_text_matrix value: (10)
intended_matrix value: O
--------------------
Coordinates: (2, 10)
combined_text_matrix value: O
intended_matrix value: X
--------------------
Coordinates: (2, 11)
combined_text_matrix value: O
intended_matrix value: (10)
--------------------
Coordinates: (4, 4)
combined_text_matrix value: X
intended_matrix value: (13G)
--------------------
Coordinates: (4, 5)
combined_text_matrix value: X
intended_matrix value: O
--------------------
Coordinates: (4, 6)
combined_text_matrix value: X
intended_matrix value: O
--------------------
Coordinates: (5, 4)
combined_text_matrix value: X
intended_matrix value: O
--------------------
Coordinates: (5, 7)
combined_text_matrix value: (XG)
intended_matrix value: X
--------------------
Coordinates: (6, 4)
combined_text_matrix value: X
intended_matrix value: O
--------------------
Coordinates: (6, 7)
combined_text_matrix value: (X)
intended_matrix value: X
--------------------
Coordinates: (7, 5)
com

TBC...

The final bit attempts to feed the summarized text data above into GPT-4 and prompt it to resolve the puzzle and return the characters that correspond to the green letters

Further development could be to add the 5 options from the Waitrose website and let GPT decide what the most likely fit is O...O

Another thing would be to directly fetch the right page from the weekly magazine update,
and then automatically submit the solution and autofill the data on the form